In [ ]:
new_virtual = {"internal": ["internal-22q4-56d4", 57, 78], "dmc": ["dmc-22q4-a73a", 52, 73], "public": ["public-22q4-6837", 52, 68]}
old_virtual = {"internal": "internal-22q2-097a", "dmc": "dmc-22q2-5e51", "public": "public-22q2-de04"}

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

In [ ]:
# portal = "internal"
from depmap_omics_upload import tracker as track
import pandas as pd

mytracker = track.SampleTracker()
pr_table = mytracker.add_model_cols_to_prtable(["ModelID"])
blacklist = set(['ACH-001956', 'ACH-000116', 'ACH-001957', 'ACH-001955'])

def appendLegacyCN(portal):
    new_mat = tc.get(name=new_virtual[portal][0], version=new_virtual[portal][1], file='OmicsCNGene')
    old_mat = tc.get(name=old_virtual[portal], file='CCLE_gene_cn')
    legacy_segments = tc.get(name='copy-number-5f61', version=5, file='legacy_segments')
    default_model_table = tc.get(name=new_virtual[portal][0], file='OmicsDefaultModelProfiles')
    default_mc_table = tc.get(name=new_virtual[portal][0], file='OmicsDefaultModelConditionProfiles')
    legacy_models = set(legacy_segments.DepMap_ID)

    new_models = set(new_mat.index)
    old_models = set(old_mat.index)

    missing_in_new = old_models - new_models
    missing_in_old = new_models - old_models

    to_append = [l for l in missing_in_new if (l in legacy_models and l not in blacklist)]
    print("appending " + str(len(to_append)) + " rows for " + portal)
    
#     return to_append
    rename_model_to_pr = dict()
    rename_mc_to_pr = dict()
    prs_to_append = []
    for l in to_append:
        if len(pr_table[(pr_table.ModelID == l) & (pr_table.ProfileSource == "taiga")]) == 1:
            pr = pr_table[(pr_table.ModelID == l) & (pr_table.ProfileSource == "taiga")].index[0]
            mc = pr_table.loc[pr, "ModelCondition"]
            prs_to_append.append(pr)
            rename_model_to_pr[l] = pr
            rename_mc_to_pr[mc] = pr
    
    return prs_to_append

#     to_append_df = old_mat[old_mat.index.isin(to_append)]

# #     full_mat = new_mat.append(to_append_df)

# #     full_mat.to_csv("../output/22Q4/gene_cn_with_legacy.csv")
    
#     # appending to default tables
#     app_mc = pd.DataFrame.from_dict(rename_mc_to_pr, orient='index').reset_index()
#     app_mc.columns=['ModelConditionID', "ProfileID"]
#     app_mc["ProfileType"] = "CN only"
#     full_mc_table = pd.concat([default_mc_table, app_mc], ignore_index=True)
#     full_mc_table.to_csv("../output/22Q4/mc_table_with_legacy.csv", index=False)
    
#     app_model = pd.DataFrame.from_dict(rename_model_to_pr, orient='index').reset_index()
#     app_model.columns=['ModelID', "ProfileID"]
#     app_model["ProfileType"] = "CN only"
#     full_model_table = pd.concat([default_model_table, app_model], ignore_index=True)
#     full_model_table.to_csv("../output/22Q4/model_table_with_legacy.csv", index=False)
    
    
    
#     tc.update_dataset(
#         changes_description="append legacy",
#         dataset_id=new_virtual[portal][0],
#         upload_files=[
# #             {
# #                 "path": "../output/22Q4/gene_cn_with_legacy.csv",
# #                 "name": "OmicsCNGene",
# #                 "format": "NumericMatrixCSV",
# #                 "encoding": "utf-8",
# #             },
#             {
#                 "path": "../output/22Q4/mc_table_with_legacy.csv",
#                 "name": "OmicsDefaultModelConditionProfiles",
#                 "format": "TableCSV",
#                 "encoding": "utf-8",
#             },
#             {
#                 "path": "../output/22Q4/model_table_with_legacy.csv",
#                 "name": "OmicsDefaultModelProfiles",
#                 "format": "TableCSV",
#                 "encoding": "utf-8",
#             },
#         ],
#         add_all_existing_files=True,
#     )

In [ ]:
for p in new_virtual.keys():
    OmicsProfiles = tc.get(name=new_virtual[p][0], version=new_virtual[p][2], file='OmicsProfiles')
    to_append = appendLegacyCN(p)
    profile_table = mytracker.add_model_cols_to_prtable(["ModelID"])

    prs = list(set(OmicsProfiles.ProfileID)) + list(set(to_append))
    profile_table = profile_table.loc[prs, ["ModelCondition", "ModelID", "Datatype"]]
    profile_table = profile_table[profile_table["Datatype"].isin(["rna", "wgs", "wes", "SNParray"])]
    profile_table = profile_table.rename(columns={"ModelCondition": "ModelConditionID"})
    profile_table.to_csv("../output/22Q4/omics_profiles_with_legacy.csv")
    
    tc.update_dataset(
        changes_description="append legacy",
        dataset_id=new_virtual[p][0],
        upload_files=[
            {
                "path": "../output/22Q4/omics_profiles_with_legacy.csv",
                "name": "OmicsProfiles",
                "format": "TableCSV",
                "encoding": "utf-8",
            },
        ],
        add_all_existing_files=True,
    )

In [ ]:
to_append

In [ ]:
from depmap_omics_upload import tracker as track

mytracker = track.SampleTracker()
pr_table = mytracker.add_model_cols_to_prtable(["ModelID"])

In [ ]:
len(to_append)

In [ ]:
# do the sam on seg level for latest:
new_mat = tc.get(name="cn-latest-d8d4", version=29, file='merged_gene_cn_profile')
old_mat = tc.get(name="internal-22q2-097a", file='CCLE_gene_cn')

to_append = [i for i in to_append if i not in ['ACH-001956', 'ACH-001955', 'ACH-001957', 'ACH-000116']]

to_append_df = old_mat[old_mat.index.isin(to_append)]
full_mat = new_mat.append(to_append_df)

full_mat = full_mat.rename(index=rename_model_to_pr)

full_mat.to_csv("../output/22Q4/gene_cn_with_legacy_pr.csv")

tc.update_dataset(
    changes_description="append legacy",
    dataset_id="cn-latest-d8d4",
    upload_files=[
        {
            "path": "../output/22Q4/gene_cn_with_legacy_pr.csv",
            "name": "merged_gene_cn_profile",
            "format": "NumericMatrixCSV",
            "encoding": "utf-8",
        },
    ],
    add_all_existing_files=True,
)

In [ ]:
"ACH-000580" in to_append

In [ ]:
for p in ['internal']:
    print(p + ": ")
    to_append_internal = appendLegacyCN(p)

In [ ]:
to_append_internal = ['ACH-001094', 'ACH-000600', 'ACH-000731', 'ACH-001230', 'ACH-001224', 'ACH-000229', 'ACH-001088', 'ACH-002217', 'ACH-001150', 'ACH-001087', 'ACH-001090', 'ACH-000049', 'ACH-001142', 'ACH-002378', 'ACH-001000', 'ACH-001234', 'ACH-000737', 'ACH-001018', 'ACH-001044', 'ACH-000034', 'ACH-000306', 'ACH-000071', 'ACH-001017', 'ACH-000658', 'ACH-001121', 'ACH-000282', 'ACH-001072', 'ACH-001956', 'ACH-000494', 'ACH-000300', 'ACH-000230', 'ACH-000539', 'ACH-000116', 'ACH-001045', 'ACH-000578', 'ACH-001071', 'ACH-000850', 'ACH-000931', 'ACH-001225', 'ACH-000642', 'ACH-001015', 'ACH-000016', 'ACH-001079', 'ACH-000185', 'ACH-000710', 'ACH-000742', 'ACH-001101', 'ACH-001207', 'ACH-001957', 'ACH-000064', 'ACH-000333', 'ACH-001171', 'ACH-000170', 'ACH-001042', 'ACH-000088', 'ACH-000854', 'ACH-000194', 'ACH-001037', 'ACH-001214', 'ACH-000003', 'ACH-000690', 'ACH-000299', 'ACH-001249', 'ACH-001198', 'ACH-000014', 'ACH-000923', 'ACH-000575', 'ACH-000413', 'ACH-000057', 'ACH-001047', 'ACH-001043', 'ACH-001955', 'ACH-000526', 'ACH-000904', 'ACH-000580', 'ACH-001091', 'ACH-001175']

In [ ]:
[i for i in to_append if i not in to_append_internal]

In [ ]:
len(to_append_internal)

In [ ]:
from depmap_omics_upload import tracker as track

mytracker = track.SampleTracker()
pr_table = mytracker.add_model_cols_to_prtable(["ModelID"])

In [ ]:
snps = []
rename_model_to_pr = dict()
rename_mc_to_pr = dict()
blacklist = set(['ACH-001956', 'ACH-000116', 'ACH-001957', 'ACH-001955'])
for l in to_append:
    if l not in blacklist:
        if len(pr_table[(pr_table.ModelID == l) & (pr_table.ProfileSource == "taiga")]) == 1:
            pr = pr_table[(pr_table.ModelID == l) & (pr_table.ProfileSource == "taiga")].index[0]
            mc = pr_table.loc[pr, "ModelCondition"]
            rename_model_to_pr[l] = pr
            rename_mc_to_pr[mc] = pr
            if pr_table.loc[pr, "Datatype"] == "SNParray":
                snps.append(l)
            if pr_table.loc[pr, "BlacklistOmics"]:
                print(l)


In [ ]:
len(to_append)

In [ ]:
import pandas as pd

app_mc = pd.DataFrame.from_dict(rename_mc_to_pr, orient='index').reset_index()
app_mc.columns=['ModelConditionID', "ProfileID"]
app_mc["ProfileType"] = "CN only"
pd.concat([app_mc, OmicsDefaultModelConditionProfiles], ignore_index=True)

In [ ]:
len(to_append)

## some mutation QC

In [ ]:
OmicsSomaticMutationsMatrixHotspot = tc.get(name='internal-22q4-56d4', version=23, file='OmicsSomaticMutationsMatrixHotspot')
OmicsSomaticMutationsMatrixDamaging = tc.get(name='internal-22q4-56d4', version=23, file='OmicsSomaticMutationsMatrixDamaging')
OmicsSomaticMutations = tc.get(name='internal-22q4-56d4', version=23, file='OmicsSomaticMutations')


In [ ]:
OmicsSomaticMutationsMatrixHotspot[[c for c in OmicsSomaticMutationsMatrixHotspot.columns if c.startswith("BRAF")]].value_counts()

In [ ]:
OmicsSomaticMutationsMatrixHotspot[OmicsSomaticMutationsMatrixHotspot['BRAF (673)'] == 2.0].index

In [ ]:
OmicsSomaticMutations[(OmicsSomaticMutations.DepMap_ID == "ACH-000121") & (OmicsSomaticMutations.HugoSymbol == "BRAF")][["GT", "HessDriver", "CCLEDeleterious", "LikelyLoF"]]

In [ ]:
OmicsSomaticMutationsMatrixDamaging[[c for c in OmicsSomaticMutationsMatrixDamaging.columns if c.startswith("BRAF")]].value_counts()

In [ ]:
OmicsSomaticMutationsMatrixDamaging[OmicsSomaticMutationsMatrixDamaging['BRAF (673)'] == 2.0].index

In [ ]:
from depmap_omics_upload import upload

d = upload.getPRToRelease()

In [ ]:
"PR-TOatoO" in d['internal']

In [ ]:
from depmap_omics_upload import tracker as track

mytracker = track.SampleTracker()
pr_table = mytracker.read_pr_table()

In [ ]:
pr_table[pr_table.ProfileSource.isnull()]

In [ ]:
OmicsGuideMutationsBinaryAvana_q4 = tc.get(name='internal-22q4-56d4', version=40, file='OmicsGuideMutationsBinaryAvana')
OmicsGuideMutationsBinaryAvana_dryrun = tc.get(name='internal-22q2dryrun-5ffb', version=113, file='OmicsGuideMutationsBinaryAvana')


In [ ]:
cols = [c for c in OmicsGuideMutationsBinaryAvana_q4.columns if c in OmicsGuideMutationsBinaryAvana_dryrun.columns]
OmicsGuideMutationsBinaryAvana_q4 = OmicsGuideMutationsBinaryAvana_q4[cols]

In [ ]:
len(OmicsGuideMutationsBinaryAvana_q4['ACH-001552'].compare(OmicsGuideMutationsBinaryAvana_dryrun['ACH-001552']))

In [ ]:
for c in OmicsGuideMutationsBinaryAvana_q4.columns.tolist():
    if len(OmicsGuideMutationsBinaryAvana_q4[c].compare(OmicsGuideMutationsBinaryAvana_dryrun[c])) > 0:
        print(c)

In [ ]:
OmicsDefaultModelConditionProfiles

In [ ]:
merged_gene_cn_profile = tc.get(name='cn-latest-d8d4', file='merged_gene_cn_profile')


In [ ]:
[i for i in merged_gene_cn_profile.index if not i.startswith("PR-")]

In [ ]:
"PR-h2XOKt" in merged_gene_cn_profile.index

In [ ]:
OmicsDefaultModelProfiles_internal = tc.get(name='internal-22q4-56d4', version=58, file='OmicsDefaultModelProfiles')
OmicsDefaultModelProfiles = tc.get(name='public-22q4-6837', version=53, file='OmicsDefaultModelProfiles')
OmicsDefaultModelConditionProfiles = tc.get(name='public-22q4-6837', version=53, file='OmicsDefaultModelConditionProfiles')


In [ ]:
OmicsDefaultModelProfiles_internal[OmicsDefaultModelProfiles_internal.ModelID == "ACH-000580"]

In [ ]:
OmicsDefaultModelProfiles[OmicsDefaultModelProfiles.ModelID == "ACH-000580"]

In [ ]:
OmicsSomaticMutations_q4 = tc.get(name='internal-22q4-56d4', version=58, file='OmicsSomaticMutations')


In [ ]:
"PR-T9HSIG" in OmicsDefaultModelConditionProfiles.ProfileID.tolist()

In [ ]:
OmicsDefaultModelConditionProfiles[OmicsDefaultModelConditionProfiles.ModelConditionID == "MC-000580-ymtS"]

In [ ]:
OmicsSomaticMutations = tc.get(name='internal-22q4-56d4', version=58, file='OmicsSomaticMutations')
OmicsSomaticMutations_dryrun = tc.get(name='internal-22q2dryrun-5ffb', version=113, file='OmicsSomaticMutations')


In [ ]:
OmicsSomaticMutations_dryrun = OmicsSomaticMutations_dryrun[[c for c in OmicsSomaticMutations]]

In [ ]:
OmicsSomaticMutations = OmicsSomaticMutations[OmicsSomaticMutations.DepMap_ID.isin(set(OmicsSomaticMutations_dryrun.DepMap_ID))]

In [ ]:
OmicsSomaticMutations[OmicsSomaticMutations.DepMap_ID == "ACH-002252"].reset_index(drop=True).compare(OmicsSomaticMutations_dryrun[OmicsSomaticMutations_dryrun.DepMap_ID == "ACH-002252"].reset_index(drop=True))

In [ ]:
old = OmicsSomaticMutations_dryrun[OmicsSomaticMutations_dryrun.DepMap_ID == "ACH-000786"].reset_index(drop=True)
new = OmicsSomaticMutations[OmicsSomaticMutations.DepMap_ID == "ACH-000786"].reset_index(drop=True)

In [ ]:
arxspans = set(OmicsSomaticMutations.DepMap_ID)

for i in arxspans:
    if len(OmicsSomaticMutations_dryrun[(OmicsSomaticMutations_dryrun.DepMap_ID == i) & (OmicsSomaticMutations_dryrun.HessDriver == "Y")]) != len(OmicsSomaticMutations[(OmicsSomaticMutations.DepMap_ID == i) & (OmicsSomaticMutations.HessDriver == "Y")]):
        print(i)
#     if len(OmicsSomaticMutations[OmicsSomaticMutations.DepMap_ID == i].reset_index(drop=True).compare(OmicsSomaticMutations_dryrun[OmicsSomaticMutations_dryrun.DepMap_ID == i].reset_index(drop=True))) > 0:
#         print("different " + i)

In [ ]:
OmicsSomaticMutations_dryrun[(OmicsSomaticMutations_dryrun.DepMap_ID == "ACH-000714") & (OmicsSomaticMutations_dryrun.HessDriver == "Y")]#[["Chrom", "Pos", "HugoSymbol"]]

In [ ]:
OmicsSomaticMutations[(OmicsSomaticMutations.DepMap_ID == "ACH-000714") & (OmicsSomaticMutations.HessDriver == "Y")]#[["Chrom", "Pos", "HugoSymbol"]]

In [ ]:
OmicsSomaticMutations_dryrun[OmicsSomaticMutations_dryrun.DepMap_ID == "ACH-000373"].reset_index(drop=True)

In [ ]:
OmicsSomaticMutations[OmicsSomaticMutations.DepMap_ID == "ACH-000373"].reset_index(drop=True)

In [ ]:
new_cleaned = tc.get(name='comparison-new-maf-old-maf-24b4', version=10, file='new_cleaned')


In [ ]:
new_cleaned.sample_id.value_counts()

In [ ]:
OmicsProfiles = tc.get(name='dmc-22q4-a73a', file='OmicsProfiles')
OmicsDefaultModelConditionProfiles = tc.get(name='dmc-22q4-a73a', file='OmicsDefaultModelConditionProfiles')


In [ ]:
len(set(OmicsProfiles.ModelConditionID))

In [ ]:
len(set(OmicsDefaultModelConditionProfiles.ModelConditionID))

In [ ]:
set(OmicsDefaultModelConditionProfiles.ModelConditionID) - set(OmicsProfiles.ModelConditionID)

In [ ]:
set(OmicsProfiles.ModelConditionID) - set(OmicsDefaultModelConditionProfiles.ModelConditionID)

In [ ]:
OmicsProfiles

In [ ]:
OmicsExpressionProteinCodingGenesTPMLogp1 = tc.get(name='public-22q4-6837', version=71, file='OmicsExpressionProteinCodingGenesTPMLogp1')
OmicsCNGene = tc.get(name='public-22q4-6837', version=71, file='OmicsCNGene')


In [ ]:
print(OmicsExpressionProteinCodingGenesTPMLogp1.index.tolist())

In [ ]:
set(OmicsExpressionProteinCodingGenesTPMLogp1.index.tolist()) - set(OmicsCNGene.index.tolist())

In [ ]:
OmicsAbsoluteCNGene = tc.get(name='internal-22q4-56d4', version=77, file='OmicsAbsoluteCNGene')
OmicsAbsoluteCNGene_dryrun = tc.get(name='internal-22q2dryrun-5ffb', version=113, file='OmicsAbsoluteCNGene')


In [ ]:
set(OmicsAbsoluteCNGene.index) - set(OmicsAbsoluteCNGene_dryrun.index)

In [ ]:
set(OmicsAbsoluteCNGene_dryrun.index) - set(OmicsAbsoluteCNGene.index)

In [ ]:
OmicsAbsoluteCNGene_dryrun